In [3]:
import os
import numpy as np
import scipy.io as sio
import scipy.signal
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
import tensorflow as tf

def load_data_from_mat_with_stft(path, fs=96, nperseg=30, noverlap=15):
    train_data = []
    train_labels = []
    test_data = []
    test_labels = []

    for file in os.listdir(path):
        if file.startswith('train_'):
            mat = sio.loadmat(os.path.join(path, file))
            for i in range(mat[list(mat.keys())[-2]].shape[0]):
                _, _, Zxx = scipy.signal.stft(mat[list(mat.keys())[-2]][i], fs=fs, nperseg=nperseg, noverlap=noverlap)
                train_data.append(np.abs(Zxx))
            train_labels.append(mat[list(mat.keys())[-1]])
        elif file.startswith('test_'):
            mat = sio.loadmat(os.path.join(path, file))
            for i in range(mat[list(mat.keys())[-2]].shape[0]):
                _, _, Zxx = scipy.signal.stft(mat[list(mat.keys())[-2]][i], fs=fs, nperseg=nperseg, noverlap=noverlap)
                test_data.append(np.abs(Zxx))
            test_labels.append(mat[list(mat.keys())[-1]])

    # 合并数据
    train_data = np.concatenate(train_data, axis=0)
    train_labels = np.concatenate(train_labels, axis=0)
    test_data = np.concatenate(test_data, axis=0)
    test_labels = np.concatenate(test_labels, axis=0)

    # 数据预处理
    # 根据STFT结果调整形状，并确保维度匹配模型输入层的要求
    # 此处reshape为(-1, height, width, channels)是示例，实际需要根据STFT结果调整
    train_data = np.log1p(train_data)  # 对数缩放
    test_data = np.log1p(test_data)
    train_data = np.expand_dims(train_data, axis=-1)
    test_data = np.expand_dims(test_data, axis=-1)
    # 数据归一化
    train_data = train_data.astype('float32') / np.max(train_data)
    test_data = test_data.astype('float32') / np.max(test_data)

    # 标签转换为one-hot编码
    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)

    return train_data, train_labels, test_data, test_labels

# 加载数据并应用STFT变换
path = '/Users/fuyao/Documents/GitHub/DL-based-RF-Fingerprints-Recognition/database/'
train_data_stft, train_labels, test_data_stft, test_labels = load_data_from_mat_with_stft(path)

print('Train data shape:', train_data_stft.shape)
print('Train labels shape:', train_labels.shape)
print('Test data shape:', test_data_stft.shape)
print('Test labels shape:', test_labels.shape)


Train data shape: (2016000, 206, 1)
Train labels shape: (126000, 10)
Test data shape: (224000, 206, 1)
Test labels shape: (14000, 10)
